In [1]:
import time

def timeit(f):

    def timed(*args, **kw):

        ts = time.time()
        result = f(*args, **kw)
        te = time.time()

        print('func:%r took: %2.4f sec' % (f.__name__,  te-ts))
        return result

    return timed

A function that help to visualize the optimization pathway:

In [2]:
%matplotlib notebook
def draw_path(func,path,x_min=-2,x_max=2,y_min=-2,y_max=2):
    a=np.linspace(x_min,x_max,100)
    b=np.linspace(y_min,y_max,100)
    x,y=np.meshgrid(a,b)
    z=func((x,y))
    fig,ax=plt.subplots()
    my_contour=ax.contour(x,y,z,50)
    plt.colorbar(my_contour)
    ax.plot(path[:,0],path[:,1])
    plt.show()

### Templates for algorithm you need to implement

In [3]:
from pylab import *
import numpy.linalg as LA
import numpy as np

@timeit
def steepest_descent(func,first_derivative,starting_point,stepsize,tol):
    # evaluate the gradient at starting point
    
    count=0
    visited=[]
    while LA.norm(first_derivative) > tol and count < 1e6:
        
        visited.append(starting_point)
        
        # calculate new point position
        
        new_point = starting_point - stepsize * first_derivative
        
        if func(new_point) < func(starting_point):
            # the step makes function evaluation lower - it is a good step. what do you do?
            starting_point = new_point
            first_derivative = deriv_2a(starting_point)
            stepsize *= 1.2
        else:
            # the step makes function evaluation higher - it is a bad step. what do you do?
            stepsize *= 0.5
            
        count+=1
    # return the results
    return {"x":starting_point,"evaluation":func(starting_point),"path":np.asarray(visited), "Steps": count}

In [4]:
def func_2a(starting_point):
    x = starting_point[0]
    y = starting_point[1]
    return x**4 - x**2 + y**2 + 2*x*y -2

def deriv_2a(starting_point):
    x = starting_point[0]
    y = starting_point[1]
    f_x = 4 * x**3 - 2 * x + 2 * y
    f_y = 2 * y + 2 * x
    return np.array([f_x, f_y])

starting_point = np.array([1.5,1.5])
first_derivative = deriv_2a(starting_point)
print(first_derivative)
new_point = starting_point - 0.1 * first_derivative
print(new_point)
result = steepest_descent(func_2a,first_derivative,starting_point,0.1,1e-5)
result

[13.5  6. ]
[0.15 0.9 ]
func:'steepest_descent' took: 0.0011 sec


{'x': array([-0.99999852,  0.99999607]),
 'evaluation': -2.999999999985186,
 'path': array([[ 1.5       ,  1.5       ],
        [ 0.15      ,  0.9       ],
        [-0.03162   ,  0.648     ],
        [-0.22733235,  0.47048256],
        [-0.4603766 ,  0.38644985],
        [-0.73063964,  0.41710875],
        [-0.91361447,  0.57314178],
        [-0.89067253,  0.77647099],
        [-0.89067253,  0.77647099],
        [-0.98168777,  0.81739147],
        [-0.94167921,  0.88803587],
        [-1.0240441 ,  0.91571465],
        [-1.0240441 ,  0.91571465],
        [-0.95964931,  0.94925202],
        [-1.01216804,  0.95311466],
        [-1.01216804,  0.95311466],
        [-0.98795692,  0.96627781],
        [-0.99481157,  0.9720766 ],
        [-0.99412388,  0.97937406],
        [-0.99741632,  0.98505533],
        [-0.99646125,  0.99076871],
        [-1.00111334,  0.9939261 ],
        [-1.00111334,  0.9939261 ],
        [-0.99723697,  0.99631795],
        [-1.00126535,  0.99668496],
        [-1.0012

In [5]:
from scipy.optimize import minimize

res_CG = minimize(func_2a, starting_point, method='CG',
               options={'gtol': 1e-6, 'disp': True})

Optimization terminated successfully.
         Current function value: -3.000000
         Iterations: 10
         Function evaluations: 84
         Gradient evaluations: 28


In [6]:
res_BFGS = minimize(func_2a, starting_point, method='BFGS',
               options={'gtol': 1e-6, 'disp': True})

Optimization terminated successfully.
         Current function value: -3.000000
         Iterations: 8
         Function evaluations: 27
         Gradient evaluations: 9


In [7]:
draw_path(func_2a,result["path"],x_min=-2,x_max=2,y_min=-2,y_max=2)

<IPython.core.display.Javascript object>

In [8]:
def stochastic_gradient_descent(func,first_derivative,starting_point,stepsize,tol=1e-5,stochastic_injection=1):
    '''stochastic_injection: controls the magnitude of stochasticity (multiplied with stochastic_deriv)
        0 for no stochasticity, equivalent to SD. 
        Use 1 in this homework to run SGD
    '''
    # evaluate the gradient at starting point
    
    count=0
    visited=[]
    while LA.norm(first_derivative) > tol and count < 1e5:
        if stochastic_injection>0:
            # formulate a stochastic_deriv that is the same norm as your gradient 
            deriv = first_derivative
            stochastic_deriv = np.random.random(len(starting_point))*2-1
            stochastic_deriv *= LA.norm(deriv) / LA.norm(stochastic_deriv)
            
        else:
            stochastic_deriv=np.zeros(len(starting_point))
            
        direction = -(deriv + stochastic_injection * stochastic_deriv)
        
        # calculate new point position
        new_point = starting_point + stepsize * direction
        
        if func(new_point) < func(starting_point):
            # the step makes function evaluation lower - it is a good step. what do you do?
            visited.append(starting_point)
            starting_point = new_point
            first_derivative = deriv_3(starting_point)
            stepsize *= 1.2
            
        else:
            # the step makes function evaluation higher - it is a bad step. what do you do?
            stepsize *= 0.5
        count+=1
    return {"x":starting_point,"evaluation":func(starting_point),"path":np.asarray(visited), "steps": count}

In [9]:
def func_3(starting_point):
    x = starting_point[0]
    y = starting_point[1]
    return (1-x)**2+10*(y-x**2)**2

def deriv_3(starting_point):
    x = starting_point[0]
    y = starting_point[1]
    f_x = -2 + 40 * x**3 +(2 - 40*y)*x
    f_y = 20 * (y - x**2)
    return np.array([f_x, f_y])

starting_point = np.array([-0.5,1.5])
first_derivative = deriv_3(starting_point)
print(first_derivative)
result = stochastic_gradient_descent(func_3,first_derivative,starting_point,0.1,1e-5,stochastic_injection=1)
result

[22. 25.]


{'x': array([0.99998948, 0.99997848]),
 'evaluation': 1.1300315549216173e-10,
 'path': array([[-0.5       ,  1.5       ],
        [-1.02131351,  1.81128855],
        [-1.04922833,  1.30654584],
        ...,
        [ 0.99998922,  0.9999777 ],
        [ 0.99998933,  0.99997795],
        [ 0.99998938,  0.99997798]]),
 'steps': 2412}

In [10]:
draw_path(func_3,result["path"],-1.2,1.2,-0.6,1.6)

<IPython.core.display.Javascript object>

In [11]:
result = steepest_descent(func_3,first_derivative,starting_point,0.1,1e-5)
result

func:'steepest_descent' took: 5.8237 sec


{'x': array([-0.98458137,  0.93299319]),
 'evaluation': 3.9518181129014924,
 'path': array([[-0.5       ,  1.5       ],
        [-0.5       ,  1.5       ],
        [-0.5       ,  1.5       ],
        ...,
        [-0.98458137,  0.93299319],
        [-0.98458137,  0.93299319],
        [-0.98458137,  0.93299319]]),
 'Steps': 1000000}

In [12]:
draw_path(func_3,result["path"],-1.2,1.2,-0.6,1.6)

<IPython.core.display.Javascript object>

In [13]:
def SGDM(func,first_derivative,starting_point,stepsize,momentum=0.9,tol=1e-5,stochastic_injection=1):
    # evaluate the gradient at starting point
    
    count=0
    visited=[]
    
    # Initialize the velocity
    previous_direction = np.zeros_like(starting_point)
    
    while LA.norm(first_derivative) > tol and count < 1e5:
        if stochastic_injection>0:
            # formulate a stochastic_deriv that is the same norm as your gradient
            deriv = first_derivative
            random_vector = np.random.random(len(deriv)) * 2 - 1
            stochastic_deriv = random_vector * LA.norm(deriv) / LA.norm(random_vector)
            
        else:
            stochastic_deriv=np.zeros(len(starting_point))
            
        # Update the velocity with momentum
        direction = -(deriv + stochastic_injection * stochastic_deriv)
        
        # calculate new point position
        new_point = starting_point + stepsize * (momentum * previous_direction + direction)
        
        if func(new_point) < func(starting_point):
            # the step makes function evaluation lower - it is a good step. what do you do?
            visited.append(starting_point)
            starting_point = new_point
            first_derivative = deriv_4(starting_point)
            previous_direction = momentum * previous_direction + direction
            stepsize *= 1.2
            
        else:
            # the step makes function evaluation higher - it is a bad step. what do you do?
            # if stepsize is too small, clear previous direction because we already know that is not a useful direction
            if stepsize<1e-5:
                previous_direction = previous_direction - previous_direction
            else:
                # do the same as SGD here
                stepsize *= 0.5
        count+=1
    return {"x":starting_point,"evaluation":func(starting_point),"path":np.asarray(visited), "steps":count}

In [14]:
def func_4(starting_point):
    x = starting_point[0]
    y = starting_point[1]
    return 2 * x**2 - 1.05 * x**4 + x**6 / 6 + x*y + y**2

def deriv_4(starting_point):
    x = starting_point[0]
    y = starting_point[1]
    f_x = 4 * x - 4.2 * x**3 + x**5 + y
    f_y = x + 2 * y
    return np.array([f_x, f_y])

starting_point = np.array([-1.5,-1.5])
first_derivative = deriv_4(starting_point)
print(first_derivative)
result = SGDM(func_4,first_derivative,starting_point,0.1,momentum=0.9,tol=1e-5,stochastic_injection=1)
result

[-0.91875 -4.5    ]


{'x': array([2.23922468e-06, 1.81653299e-07]),
 'evaluation': 1.0468014831298855e-11,
 'path': array([[-1.50000000e+00, -1.50000000e+00],
        [-1.09341362e+00, -7.15488767e-01],
        [-1.83954907e-01,  4.75548881e-01],
        [-1.83949766e-01,  4.75548480e-01],
        [-1.83934272e-01,  4.75535985e-01],
        [-1.83920977e-01,  4.75504905e-01],
        [-1.83913988e-01,  4.75465070e-01],
        [-1.83907332e-01,  4.75394395e-01],
        [-1.83899013e-01,  4.75318370e-01],
        [-1.83901329e-01,  4.75227897e-01],
        [-1.83921326e-01,  4.75110327e-01],
        [-1.83944326e-01,  4.74925900e-01],
        [-1.83923966e-01,  4.74680739e-01],
        [-1.83930001e-01,  4.74389566e-01],
        [-1.83875306e-01,  4.74050245e-01],
        [-1.83789420e-01,  4.73686000e-01],
        [-1.83618090e-01,  4.73171170e-01],
        [-1.83471149e-01,  4.72466381e-01],
        [-1.83360007e-01,  4.71529054e-01],
        [-1.83315118e-01,  4.70331455e-01],
        [-1.83071068e-01, 

In [15]:
draw_path(func_4,result["path"])

<IPython.core.display.Javascript object>